In [ ]:
#|default_exp api_clients

In [ ]:
# Public helpers for geocoding, routing and parking.
# nbdev exports everything to **DataTalks.api**.


In [ ]:
#|export
from __future__ import annotations

# stdlib
from dataclasses import dataclass
from datetime import datetime
import datetime as dt
import os
from typing import List, Literal, Optional

# 3rd-party
import httpx
import pydantic
import polyline


In [ ]:
#|export
# ───────────────────────────────────────────────────────────────
# Constants
# ───────────────────────────────────────────────────────────────
OSRM_URL = "https://router.project-osrm.org/route/v1"        # /driving /cycling /walking
ORS_URL  = "https://api.openrouteservice.org/v2/directions"  # /cycling-regular …


In [ ]:
#
def ping() -> str:
    "Quick *kernel-alive* helper."
    return f"Kernel OK at {datetime.now():%H:%M:%S}"


In [ ]:
#|export
class Point(pydantic.BaseModel):
    "WGS-84 point; note `.ll()` returns **lon,lat** (OSRM order)."
    lat: float
    lon: float
    def ll(self) -> str: return f"{self.lon},{self.lat}"


In [ ]:
#|export
# Generic types
Mode = Literal["car", "bike", "pt", "walk"]

@dataclass
class Step:
    instruction: str
    distance_m : float
    duration_s : float

@dataclass
class Route:
    mode       : Mode
    distance_m : float
    duration_s : float
    geometry   : List[Point]
    steps      : List[Step]


In [ ]:
#|export
# ───────────────────────────────────────────────────────────────
# Geocoding – Nominatim
# ───────────────────────────────────────────────────────────────
async def geocode(query: str) -> Point:
    """Return the first OpenStreetMap match for *query*."""
    params = {"q": query, "format": "json", "limit": 1}
    async with httpx.AsyncClient(timeout=10) as cx:
        r = await cx.get("https://nominatim.openstreetmap.org/search", params=params)
    j = r.json()[0]
    return Point(lat=float(j["lat"]), lon=float(j["lon"]))


In [ ]:
#|export
# ───────────────────────────────────────────────────────────────
# OSRM demo instance
# ───────────────────────────────────────────────────────────────
async def _osrm_route(profile: str, src: Point, dst: Point) -> Route:
    """Low-level helper around the public OSRM server."""
    url    = f"{OSRM_URL}/{profile}/{src.ll()};{dst.ll()}"
    params = {"overview": "full", "steps": "true", "geometries": "polyline"}
    async with httpx.AsyncClient(timeout=10) as cx:
        r = await cx.get(url, params=params)
    r.raise_for_status()
    data = r.json()["routes"][0]

    coords = [Point(lat=lat, lon=lon) for lon, lat in polyline.decode(data["geometry"])]

    def _instr(s):
        m = s["maneuver"]
        parts = [m["type"], m.get("modifier"), s.get("name")]
        return " ".join(p for p in parts if p)

    steps = [Step(_instr(s), s["distance"], s["duration"])
             for leg in data["legs"] for s in leg["steps"]]

    mode = {"driving": "car", "cycling": "bike", "walking": "walk"}[profile]

    return Route(mode, int(data["distance"]), int(data["duration"]), coords, steps)



In [ ]:
#|export
async def osrm_car(src: Point, dst: Point) -> Route:
    "Route for profile **driving** (car)."
    return await _osrm_route("driving", src, dst)


In [ ]:
#|export
async def osrm_walk(src: Point, dst: Point) -> Route:
    "Route for profile **walking** (foot)."
    return await _osrm_route("walking", src, dst)


In [ ]:
#|export
# ───────────────────────────────────────────────────────────────
# OpenRouteService – cycling
# ───────────────────────────────────────────────────────────────
async def ors_bike(src: Point, dst: Point, profile: str = "cycling-regular") -> Route:
    api_key = os.getenv("ORS_KEY")
    if not api_key:
        raise RuntimeError("ORS_KEY environment variable missing")

    body = {
        "coordinates": [[src.lon, src.lat], [dst.lon, dst.lat]],
        "geometry": True,
        "instructions": True,
    }
    headers = {"Authorization": api_key}

    async with httpx.AsyncClient(timeout=15) as cx:
        r = await cx.post(f"{ORS_URL}/{profile}", json=body, headers=headers)
    r.raise_for_status()
    feat  = r.json()["features"][0]
    props = feat["properties"]["summary"]

    coords = [Point(lat=lat, lon=lon)
              for lon, lat in polyline.decode(feat["geometry"])]

    steps = [Step(s["text"], s["distance"], s["duration"])
             for s in feat["properties"]["segments"][0]["steps"]]

    return Route("bike", props["distance"], props["duration"], coords, steps)


In [ ]:
#|export
# ───────────────────────────────────────────────────────────────
# Digitransit GTFS-v2 (public transport) – fastest single itinerary
# ───────────────────────────────────────────────────────────────
_DT_URL = "https://api.digitransit.fi/routing/v2/hsl/gtfs/v1"
_HEADERS = {
    "digitransit-subscription-key": os.getenv("DIGITRANSIT_KEY", "ee36f515bc2d426fa25325dd51734266"),
    "Content-Type": "application/json",
}

_PLAN_Q = """
query Plan($orig: PlanLabeledLocationInput!, $dest: PlanLabeledLocationInput!) {
  planConnection(origin: $orig, destination: $dest, first: 1) {
    edges { node {
      duration walkDistance
      legs {
        mode distance duration
        from { name lat lon }
        to   { name lat lon }
        legGeometry { points }
      }
    } }
  }
}"""

async def digitransit_pt(
    src: Point,
    dst: Point,
    *,
    dataset: str = "hsl",
) -> Route:
    """Fastest PT itinerary between *src* and *dst* via Digitransit Routing v2."""
    url = _DT_URL.replace("hsl", dataset)

    variables = {
        "orig": {"location": {"coordinate": {"latitude": src.lat, "longitude": src.lon}}},
        "dest": {"location": {"coordinate": {"latitude": dst.lat, "longitude": dst.lon}}},
    }

    async with httpx.AsyncClient(timeout=20) as cx:
        r = await cx.post(url, json={"query": _PLAN_Q, "variables": variables}, headers=_HEADERS)
    r.raise_for_status()
    body = r.json()
    if "errors" in body:
        raise RuntimeError(body["errors"][0]["message"])

    node = body["data"]["planConnection"]["edges"][0]["node"]

    geom: List[Point] = []
    steps: List[Step] = []
    for leg in node["legs"]:
        pts = [Point(lat=lat, lon=lon)
               for lon, lat in polyline.decode(leg["legGeometry"]["points"])]
        if geom and pts: pts = pts[1:]      # avoid duplicate joint
        geom.extend(pts)
        steps.append(
            Step(f"{leg['mode'].title()} {leg['from']['name']} → {leg['to']['name']}",
                 leg["distance"], leg["duration"])
        )

    dist_m = node["walkDistance"] + sum(l["distance"] for l in node["legs"] if l["mode"] != "WALK")

    return Route("pt", dist_m, node["duration"], geom, steps)


In [ ]:
#|export
# ───────────────────────────────────────────────────────────────
# HSL Park-&-Ride live status
# ───────────────────────────────────────────────────────────────
HSL_PR_BASE = "https://p.hsl.fi/api/v1"
DEFAULT_FACILITY = 285                        # Urheilupuisto

async def hsl_parking_status(facility_id: int = DEFAULT_FACILITY) -> dict:
    """Return live vacancy dict for one HSL Park-&-Ride facility."""
    url = f"{HSL_PR_BASE}/facilities/{facility_id}/status.json"
    async with httpx.AsyncClient(timeout=10) as cx:
        r = await cx.get(url)
    r.raise_for_status()
    j = r.json()
    ts = dt.datetime.fromisoformat(j["timestamp"]).astimezone(dt.timezone.utc)
    j["timestamp"] = ts.isoformat(timespec="seconds")
    return j


In [ ]:
#|export
# ───────────────────────────────────────────────────────────────
# Fintraffic LIIPI parking API – thin wrapper
# ───────────────────────────────────────────────────────────────
_FT = "https://parking.fintraffic.fi/api/v1"
_DEFAULT_APP_ID = "datatalks-bot"

class UtilRow(pydantic.BaseModel):
    facilityId      : int
    capacityType    : str
    usage           : str
    timestamp       : str
    spacesAvailable : int
    capacity        : int
    openNow         : bool

class Facility(pydantic.BaseModel):
    id            : int
    name          : dict
    operatorId    : Optional[int] = None
    builtCapacity : dict[str, int] = {}
    status        : str

class FintrafficParking:
    """Minimal async client for the Fintraffic Parking REST API."""
    def __init__(self, *, token: str | None = None, app_id: str = _DEFAULT_APP_ID, timeout: float = 10):
        self._tok  = token
        self._head = {"Digitraffic-User": app_id}
        if token: self._head["Authorization"] = f"Bearer {token}"
        self._timeout = timeout

    async def _get(self, path: str, **params):
        async with httpx.AsyncClient(timeout=self._timeout) as cx:
            r = await cx.get(f"{_FT}{path}", headers=self._head, params=params)
        r.raise_for_status()
        return r.json()

    async def facility(self, fid: int) -> Facility:
        return Facility(**await self._get(f"/facilities/{fid}"))

    async def utilization(self, fid: int, *, capacity_type="CAR", usage="PARK_AND_RIDE") -> UtilRow:
        rows = [UtilRow(**u) for u in await self._get(f"/facilities/{fid}/utilization")]
        try:
            return next(r for r in rows if r.capacityType == capacity_type and r.usage == usage)
        except StopIteration:
            raise ValueError("No matching utilization row found")


In [ ]:
#|export
# Convenience wrapper: one JSON for the chat-bot
METRO_AREENA_ID = 285          # Urheilopuisto P+R

async def parking_status(
    facility_id: int = METRO_AREENA_ID,
    *,
    capacity_type: str = "CAR",
    usage: str = "PARK_AND_RIDE",
    client: FintrafficParking | None = None,
) -> dict:
    """Return live vacancy (spaces, capacity, timestamp_utc, openNow)."""
    api = client or FintrafficParking()
    row = await api.utilization(facility_id, capacity_type=capacity_type, usage=usage)
    ts_utc = (dt.datetime.fromisoformat(row.timestamp)
              .astimezone(dt.timezone.utc)
              .isoformat(timespec="seconds"))
    return {
        "capacity":        row.capacity,
        "spacesAvailable": row.spacesAvailable,
        "timestamp_utc":   ts_utc,
        "openNow":         row.openNow,
    }
